In [ ]:
# Vehicle Routing Problem with Time Windows
from flowty import Model, xsum
from or_datasets import vrp_rep

import boto3
import json

In [ ]:
#get json file input_data
bucket_name = 'flowty-sagemaker'
json_file = 'input/inputdata.json'

In [ ]:
#initiate s3 session
S3_client = boto3.client('s3',
)

In [ ]:
#get json file
response = S3_client.get_object(Bucket=bucket_name, Key=json_file)
content = response['Body']
jsonObject = json.loads(content.read())

In [ ]:
#assign alias
keys = jsonObject.keys()
iter_keys = iter(keys)
alias = next(iter_keys)

In [ ]:
#assign instance_name
values = jsonObject.values()
iter_values = iter(values)
instance_name = next(iter_values)

In [ ]:
#
bunch = vrp_rep.fetch_vrp_rep(alias, instance=instance_name)
name, n, E, c, d, Q, t, a, b, x, y = bunch["instance"]

In [ ]:
m = Model()

In [ ]:
# one graph, it is identical for all vehicles
g = m.addGraph(obj=c, edges=E, source=0, sink=n - 1, L=1, U=n - 2, type="B")

In [ ]:
# adds resources variables to the graph.
# travel time and customer tine windows
m.addResourceDisposable(
    graph=g, consumptionType="E", weight=t, boundsType="V", lb=a, ub=b, name="t"
)

In [ ]:
# demand and capacity
m.addResourceDisposable(
    graph=g, consumptionType="V", weight=d, boundsType="V", lb=0, ub=Q, name="d"
)

In [ ]:
# set partition constriants
for i in range(n)[1:-1]:
    m += xsum(x * 1 for x in g.vars if i == x.source) == 1

In [ ]:
# packing set
for i in range(n)[1:-1]:
    m.addPackingSet([x for x in g.vars if i == x.source])

In [ ]:
status = m.optimize()
print(f"ObjectiveValue {m.objectiveValue}")

In [ ]:
# get the variable values
for var in m.vars:
    if var.x > 0:
        print(f"{var.name} = {var.x}")

In [ ]:
### output variable_values as json file in s3 bucket subfolder###

#create dict populate dict
variable_values = {}
for var in m.vars:
    if var.x > 0:
        variable_values[var.name] = var.x

data = {"objective value":m.objectiveValue, "variable values":variable_values}

In [ ]:
#save dict
with open(name+'.json', 'w') as fp:
    json.dump(data, fp, indent=4)

#upload json object to s3 bucket in output subfolder
bucket_name = "flowty-sagemaker"
key = name+".json"  #filename

S3_client.put_object(
     Body=str(json.dumps(data, indent=4)),
     Bucket=bucket_name,
     Key="output/"+key
)

In [ ]:
### visualization and save in s3 bucket ###
import math
import networkx
import matplotlib
import matplotlib.pyplot as plt

edges = [x.edge for x in g.vars if not math.isclose(x.x, 0, abs_tol=0.001)]
gx = networkx.DiGraph()
gx.add_nodes_from([i for i in range(n)])
gx.add_edges_from(edges)
pos = {i: (x[i], y[i]) for i in range(n)} # for lists of x,y coordinates
# pos = networkx.spring_layout(gx) # alternative layout
networkx.draw_networkx_nodes(gx, pos, nodelist=gx.nodes)
networkx.draw_networkx_labels(gx, pos, labels={i: i for i in gx.nodes})
networkx.draw_networkx_edges(gx, pos, nodelist=gx.edges)
# plt.show() # if gui backend is supported
plt.savefig("mygraph.png")

#upload visualization to s3 bucket in output subfolder
S3_client.upload_file(
     "mygraph.png",
     bucket_name,
     "output/"+"mygraph.png"
)